In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [ ]:
assessor = pd.read_csv('../data/Assessor_s_Property_Database_FY14-FY18.csv')

# Create Garage Labels

In [ ]:
assessor['MBL'] = assessor.MAP.astype(str) +"-"+ assessor.BLOCK.astype(str) +"-"+  assessor.LOT.astype(str) 

In [ ]:
garage_df = (
    assessor
    .groupby('MBL')['HAS GARAGE']
    .apply(list)
    .apply(lambda l: 1 if 'Y' in l else 0)
    .reset_index()
    .rename({'HAS GARAGE':'HAS_GARAGE'}, axis =1)
)

garage_df = garage_df[garage_df.HAS_GARAGE == 1]

In [ ]:
garage_df.to_csv('../data/garage.csv')

# Clean Assessor 

In [ ]:
assessor = assessor[assessor.FISCAL_YEAR == 2018]

In [ ]:
unit_addr = assessor['STREET'].apply(lambda x: x.strip().lower())
owner_addr = assessor['OWNER ADD'].apply(lambda x: ' '.join(str(x).split(' ')[1:]).strip().lower())
assessor['OWNER_IS_TENANT'] = unit_addr == owner_addr

In [ ]:
feature_clms = [
    'EXEMPT AMT', 
    'TAX VALUE', 
    'ROOF STRUCTURE DESCRIP',
    'ROOF COVER DESCRIP', 
    'INT WALL 1 DESCRIP',
    'HEAT TYPE',
    'FUEL TYPE',
    'AC TYPE',
    'GRADE DESCRIP',
    'IMPROVE VAL',
    'MBL'
]
feature_clms_clean = [clm.replace(' ', '_') for clm in  feature_clms]

In [ ]:
replacement_clms = dict(zip(feature_clms, feature_clms_clean))

In [ ]:
assessor = (
    assessor
    .rename(replacement_clms, axis = 1)
    [feature_clms_clean]
)

In [ ]:
vc = assessor.INT_WALL_1_DESCRIP.value_counts()
assessor.INT_WALL_1_DESCRIP.apply(lambda x: vc.loc[x] if type(x) == str else np.NaN)

In [ ]:
assessor.sort_values('IMPROVE_VAL', ascending=False).drop_duplicates(subset = ['MBL'])

In [ ]:
assessor.to_csv('../data/assessor_clean.csv')